In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from time import time
from tensorflow.python.keras.callbacks import TensorBoard
import warnings
import cv2
warnings.filterwarnings("ignore")

def myFunc(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return gray

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())  
model.add(Dense(128))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# COMPILE
tb = TensorBoard(log_dir="logs/{}".format(time()))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Using TensorFlow backend.
C:\Users\datta\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\datta\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\datta\AppData\Local\Continuum\anaconda3\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [2]:
batch_size = 8
from keras.preprocessing.image import ImageDataGenerator
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)


In [3]:
train_generator = train_datagen.flow_from_directory(
        r'C:\Users\datta\Desktop\tfidf\lowfps\train',  # this is the target directory
        target_size=(64,64),  # all images will be resized to 300x300
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r'C:\Users\datta\Desktop\tfidf\lowfps\validation',
        target_size=(64,64),
        batch_size=batch_size,
        class_mode='binary')

Found 229 images belonging to 2 classes.
Found 95 images belonging to 2 classes.


In [4]:
model.fit_generator(
        train_generator,
        steps_per_epoch=250 // batch_size  ,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=50 // batch_size)


Epoch 1/20
31/31 [==============================] - 12s 392ms/step - loss: 0.4102 - accuracy: 0.7837 - val_loss: 0.0583 - val_accuracy: 0.9792
Epoch 2/20
31/31 [==============================] - 10s 316ms/step - loss: 0.1371 - accuracy: 0.9633 - val_loss: 0.0729 - val_accuracy: 0.9787
Epoch 3/20
31/31 [==============================] - 9s 302ms/step - loss: 0.0709 - accuracy: 0.9714 - val_loss: 0.0497 - val_accuracy: 1.0000
Epoch 4/20
31/31 [==============================] - 10s 333ms/step - loss: 0.1373 - accuracy: 0.9421 - val_loss: 0.6737 - val_accuracy: 0.9787
Epoch 5/20
31/31 [==============================] - 9s 294ms/step - loss: 0.0728 - accuracy: 0.9879 - val_loss: 0.0136 - val_accuracy: 0.9792
Epoch 6/20
31/31 [==============================] - 9s 293ms/step - loss: 0.0566 - accuracy: 0.9752 - val_loss: 0.0045 - val_accuracy: 1.0000
Epoch 7/20
31/31 [==============================] - 10s 337ms/step - loss: 0.0319 - accuracy: 0.9878 - val_loss: 0.0048 - val_accuracy: 1.0000
E

In [5]:
#model.save_weights('50_epochs.hdf5')  # always save your weights after training or during training
model_json = model.to_json()
with open("20_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("20_model.h5")
print("Saved model to disk")

Saved model to disk
